In [142]:
import warnings
warnings.filterwarnings("ignore")

from IPython.display import display_html
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import random
import requests
import json
import os
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA


%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.style.use("seaborn-muted")

train_fraction = 0.05
test_fraction = 0.3
max_features = 2000
fast = True

In [143]:
df = pd.read_csv("nlp-dataset/train.csv")

In [144]:
df.head(5).T

0  \
id                                                  1008537883   
title                                             2010 Audi A3   
description  A white in colour audi a3 tfsi in good conditi...   
category                                        Cars & Bakkies   

                                                             1  \
id                                                  1009127328   
title                                 1 tb internal harddrives   
description  Internal harddrives for sale 1tb 3.5 inch for ...   
category                       Computer Hardware & Accessories   

                                                             2  \
id                                                  1008982705   
title                   3× Mercedes benz Actros 26:40 for sale   
description  Make     Mercedes benz Actros 26:40\nModel    ...   
category                          Trucks & Commercial Vehicles   

                                                             3  \
id                                                  1006975280   
title                              Samsung S7 Edge (Rose Pink)   
description  Selling my Samsung S7 Edge Rose Pink colour ve...   
category                                           Cell Phones   

                                                             4  
id                                                  1008950745  
title                                     2TB Desktop Hardrive  
description  2TB Desktop Hardrive for a bargain \n\nCall or...  
category                       Computer Hardware & Accessories

In [145]:
df.category.unique()

array(['Cars & Bakkies', 'Computer Hardware & Accessories',
       'Trucks & Commercial Vehicles', 'Cell Phones',
       'Businesses for Sale', 'Furniture & Decor', 'Tools & DIY',
       'Houses & Flats for rent', 'Art, Collectibles & Rare Items',
       'Car Parts & Accessories', 'Land', 'Clothing & Shoes',
       'Rooms for rent & Shared', 'Gym & Fitness', 'TV, Audio & Visual',
       'Prams, Cots & Equipment', 'iPads & Tablets',
       'Feeds, Supplements & Seeds', 'Gaming & Consoles',
       'Musical Instruments', 'Homeware & Appliances',
       'Toys, Games & Remote Control', 'Motorcycles & Scooters',
       'Dogs & Cats', 'Construction & Home Improvement',
       'Outdoor & Sports Equipment', 'Business & Industrial Equipment',
       'CVs & Resumes', 'Health, Beauty & Cosmetics',
       'Jewellery & Accessories', 'Computers & Laptops',
       'Boats & Aviation', 'Garden & Braai',
       'Farming Equipment & Vehicles', 'Other Services', 'Bicycles',
       'Community Announcements'

In [146]:
df.category.value_counts()

Cars & Bakkies                           164468
Cell Phones                               50316
Furniture & Decor                         45247
Car Parts & Accessories                   43378
Homeware & Appliances                     29019
Motorcycles & Scooters                    24225
Gaming & Consoles                         20988
TV, Audio & Visual                        20688
Computers & Laptops                       18207
Outdoor & Sports Equipment                16991
Tools & DIY                               14611
Houses & Flats for rent                   12324
Computer Hardware & Accessories           10634
Prams, Cots & Equipment                   10009
Dogs & Cats                                7643
Clothing & Shoes                           7456
Art, Collectibles & Rare Items             7263
Trucks & Commercial Vehicles               6963
Garden & Braai                             6354
Bicycles                                   6037
Business & Industrial Equipment         

In [147]:
test = pd.read_csv("nlp-dataset/test.csv")

In [149]:
test.head(5)

id                                              title  \
0  1008387829                                      Massage Chair   
1  1007542388  Acer Aspire AX3910 PC desktop with windows 10 Pro   
2  1009234002                                        Beach buggy   
3  1007888574  Rest Assured Double mattress for sale great co...   
4  1007436437              2008 Volkswagen Polo 1.9 TDi Highline   

                                         description  
0  Electronic Massage Chair. Very good condition....  
1  This is the best PC for any student its in gre...  
2       Hy is nie voledig nie en het nie papiere nie  
3                          Very great condition firm  
4  Factory Features\r\n\r\n- ABS\r\n- Airbags\r\n...

In [150]:
df.isnull().sum()

id              0
title           0
description    14
category        0
dtype: int64

In [151]:
df.fillna("", inplace=True)

In [152]:
df["full_text"] = df.title + " " + df.description

In [153]:
encoder = LabelEncoder().fit(df.category)
df["category_id"] = encoder.transform(df.category)

In [154]:
train_df = df.sample(frac=train_fraction).reset_index(drop=True)
test_df = df[~df.index.isin(train_df.index)].reset_index(drop=True)

In [155]:
text_column = "full_text"
vec = TfidfVectorizer(max_features=max_features, ngram_range=(1, 3), analyzer="word", 
                      stop_words="english", norm="l2", token_pattern=r"(?u)\b\w+\b").fit(train_df[text_column])
assert len(vec.vocabulary_) == max_features

In [156]:
train_dataset = pd.DataFrame(vec.transform(train_df.loc[:,text_column]).todense(), columns=vec.vocabulary_)

In [157]:
test_dataset = pd.DataFrame(vec.transform(test_df.loc[:,text_column]).todense(), columns=vec.vocabulary_)

In [158]:
def model_score(actual, predicted):
    return 1.0 * np.sum((predicted == actual)) / actual.size

Bag of Word Model

---

In [159]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.embeddings import Embedding
from keras.layers.pooling import GlobalAveragePooling1D
from sklearn.preprocessing import MinMaxScaler
from keras.layers.advanced_activations import PReLU
from keras.optimizers import SGD

In [160]:
def OneHotEncoding(df, category=55):
    vector = np.zeros([df.shape[0], category])
    for i in xrange(train_df.shape[0]):
        vector[i][train_df.ix[i, 'category_id']] = 1
    
    return vector

In [ ]:
# losses: binary_crossentropy, categorical_crossentropy
def get_bow_model(input_shape, output_dim):
    model = Sequential()
    model.add(Dense(768, input_shape=input_shape))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(output_dim))
    model.add(Activation('sigmoid'))
    model.compile(
        loss='categorical_crossentropy',
        optimizer="Adadelta",
        metrics=['accuracy'])
    return model

In [ ]:
pred = None
label = OneHotEncoding(train_df, )
model = get_bow_model((train_dataset.shape[1],), 55)
model.fit(train_dataset.values, 
          label,
          batch_size=32, nb_epoch=20, verbose=3)
pred = model.predict(test_dataset.values) if pred is None else pred + model.predict(test_dataset.values)

Epoch 1/20
Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20
Epoch 6/20
Epoch 7/20
Epoch 8/20
Epoch 9/20
Epoch 10/20


In [ ]:
prediction = pred.argmax(axis=1)
model_score(test_df.category_id.values, prediction)

In [ ]:
test_df = test
test_df.fillna("", inplace=True)
test_df["full_text"] = test_df.title + " " + test_df.description

In [ ]:
test_dataset = pd.DataFrame(vec.transform(test_df.loc[:,text_column]).todense(), columns=vec.vocabulary_)
pred = model.predict(test_dataset.values)
prediction = pred.argmax(axis=1)

In [ ]:
test_df["category_id"] = prediction
test_df["category"] = encoder.inverse_transform(prediction)
test_df = test_df[["id", "category"]]
test_df.to_csv("output.csv", index=False)